In [ ]:
imp = open('/home/lindb/pythonimports.py').read()
exec(imp)
from __future__ import division
%matplotlib inline 

In [ ]:
mDIR ='/home/lindb/teakettle/postmoran/chloro'
if not op.exists(mDIR):
    mkdir(mDIR)

# for all teakettle - chloro only

### Put into R
```R
source('~/imports.R')
library(PopGenReport)
library(adegenet)

moran     <- read.csv('~/Desktop/moran_infile_final_jan_wconecounts.txt',sep='\t')
simpmoran <- moran[seq(1,nrow(moran),2),] # for spatial stuff

### name some stuff
chloro <- c('pc10','pt71936','pt87268')
nuc    <- c('rps12','rps2','rps39','rps50')
loci   <- c(nuc,chloro)

### make a blank df to fill in later
popgen <- matrix(0,ncol=len(chloro)+1,nrow=luni(moran$samp))
popgen <- data.frame(popgen)
colnames(popgen) <- c('pop',chloro)
rownames(popgen) <- uni(moran$samp)
head(popgen)

### get genotype
getgt = function(type,df,locus){
    rownames(df) <- 1:nrow(df)
    stopifnot(nrow(df)==1)
    if (type == "nuc"){
        one = df[1,locus]
        two = df[1,sprintf("%s_2",locus)]
        combined = sprintf("%s:%s",one,two)
    } else {
        combined = sprintf("%s:%s",df[1,locus],df[1,locus])
    }
    return(combined)
}

# make function to make df
getdf = function(treatment='tx'){
    if (!treatment %in% c('tx','plot','tfk')){
        stop(cat(sprintf("treatment %s not in %s",treatment,c('tx','plot','single'))))
    }
    # make blank df
    popgen <- matrix(0,ncol=len(loci)+1,nrow=luni(moran$samp))
    popgen <- data.frame(popgen)
    colnames(popgen) <- c('pop',loci)
    rownames(popgen) <- uni(moran$samp)

    # fill in blank df
    for (samp in uni(moran$samp)){
        df <- moran[which(moran$samp == samp),]

        plot = strsplit(samp,split="_")[[1]][1]
        tx   = substr(plot,1,2)
        if (treatment=='tx'){
            popgen[samp,'pop'] <- tx
        } else if (treatment == 'plot'){
            popgen[samp,'pop'] <- plot
        } else {
            popgen[samp,'pop'] <- 'teakettle'
        }
        for (locus in loci){
            type = NULL
            if (locus %in% nuc){
                type = 'nuc'
            } else {
                type = 'chloro'}
            if (df[1,locus] != 0){
                popgen[samp,locus] = getgt(type,df[1,],locus)
            } else if (df[2,locus] != 0) {
                popgen[samp,locus] = getgt(type,df[2,],locus)
            } else {
                popgen[samp,locus] = NA}}}
    return(popgen)}

### make dfs with either plot or treatment
popgen.tfk  <- getdf(treatment='tfk')  # one single pop
head(popgen.tfk)
# df with only loci as columns
pg.tfk <- popgen.tfk[,names(popgen.tfk[2:len(names(popgen.tfk))])]
head(pg.tfk)

### convert to class(genind)
tfkpop  <- popgen.tfk$pop
gentfk  <- df2genind(pg.tfk,ploidy=2,sep=":",pop=tfkpop)

### calc gen dist
gd <- gd.smouse(gentfk)
gdmat <- as.matrix(gd)
dim(gdmat)
gdmat[1:5,1:5]

### get euclidian distance
distmat <- readRDS('~/fragment/postmoran/final/distmat.RDS')
dmat <- distmat[which(rownames(distmat) %in% rownames(gdmat)),which(colnames(distmat) %in% colnames(gdmat))]
all(rownames(dmat) == rownames(gdmat))
all(colnames(dmat) == colnames(gdmat))
luni(rownames(dmat)) == nrow(gdmat)
luni(rownames(gdmat)) == nrow(dmat)
all(dim(dmat) == dim(gdmat))

### get spatial autocorrelation
bins = ceiling(.max(dmat)/10) # = 173
sauto <- spautocor(gdmat,dmat,bins = 173)
range(sauto$r)

### save environment
cd('~/fragment/postmoran/all_tfk-chloro')
rds()
```

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/chloro/final'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
# copy from desktop to godel
# scp ~/fragment/postmoran/all_tfk-chloro/*RDS godel:/home/lindb/teakettle/postmoran/chloro/final

### multiprocess null confidence intervals for spatial autocorrelation

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/chloro/null/rfiles'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/chloro/null/shfiles'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
if not op.exists('/home/lindb/teakettle/postmoran/chloro/null/outfiles/'):
    mkdir('/home/lindb/teakettle/postmoran/chloro/null/outfiles/')

In [ ]:
for i in range(999):
    text = '''
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/chloro/final')

# load dependencies
getrds()

spautocor <- function (gen.m, eucl.m, shuffle = FALSE, bins = 10) 
{
    gd <- gen.m
    ed <- eucl.m
    if (shuffle == TRUE) {
        gdd <- as.dist(gd)
        gdsample <- sample(1:length(gdd), length(gdd))
        gd[lower.tri(gd)] <- gdd[gdsample]
        gd[upper.tri(gd)] <- gdd[gdsample]
        diag(gd) <- 0
    }
    cdmat <- function(gd) {
        dimen <- nrow(gd)
        sgd <- sum(gd, na.rm = TRUE)
        cscd <- matrix(colSums(gd, na.rm = TRUE), dimen, dimen)
        rscd <- matrix(rowSums(gd, na.rm = TRUE), dimen, dimen, 
            byrow = TRUE)
        cd <- 0.5 * (-gd + 1/dimen * (cscd + rscd) - 1/dimen^2 * 
            (sgd))
        cd
    }
    cd <- cdmat(gd)
    ed[upper.tri(ed)] <- NA
    diag(ed) <- NA
    r <- NA
    distance <- NA
    N <- NA
    steps <- signif(diff(range(ed, na.rm = TRUE))/bins, 4)
    for (d in 1:bins) {
        index <- which(ed <= d * steps & ed > (d - 1) * steps, 
            arr.ind = TRUE)
        cx <- sum(cd[index])
        cxii <- sum(diag(cd)[index[, 1]])
        cxjj <- sum(diag(cd)[index[, 2]])
        r[d] <- 2 * cx/(cxii + cxjj)
        distance[d] <- steps * d
        N[d] <- length(index)
    }
    if (shuffle == FALSE) 
        res <- data.frame(bin = distance, N = N, r = r)
    else res <- data.frame(r = r)
    res
}

# get null value
shuff <- spautocor(gdmat,dmat,bins=bins,shuffle=T)
filE = sprintf('/home/lindb/teakettle/postmoran/chloro/null/outfiles/shuff_%%s.RDS',sprintf('%%03d',%s))
saveRDS(shuff,filE)

''' % (i)
    filE = '/home/lindb/teakettle/postmoran/chloro/null/rfiles/null_%s.R' % str(i).zfill(3)
    with open(filE,'w') as o:
        o.write("%s" % text)
    o.close()

In [ ]:
files = fs('/home/lindb/teakettle/postmoran/chloro/null/rfiles/')
math.ceil(len(files)/195)

In [ ]:
files = fs('/home/lindb/teakettle/postmoran/chloro/null/rfiles/')

fcount  = 0
shcount = 0
rcount  = 0
for f in files:
    num = op.basename(f).split(".R")[0].split("_")[1]
    if rcount == 0:
        text = '''#!/bin/bash
#$ -N shuff%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s
''' % (num,
       op.dirname(f),
       f)
    else:
        newtext = '''
cd %s
R --no-save < %s
''' % (op.dirname(f),
       f)
        text = text + newtext
    fcount += 1
    rcount += 1
    
    if rcount == 6 or fcount == len(files):
        filE = '/home/lindb/teakettle/postmoran/chloro/null/shfiles/shuff_%s.sh' % str(shcount).zfill(3)
        with open(filE,'w') as o:
            o.write("%s" % text)
        rcount = 0
        shcount += 1

In [ ]:
# qsub files
# !cd /home/lindb/teakettle/postmoran/chloro/null/shfiles/
# !ls *sh | parallel qsub {}

### combine null runs

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/chloro/null/outfiles/'
text = '''
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/chloro/')

sauto <- readRDS('/home/lindb/teakettle/postmoran/chloro/final/sauto.RDS')

# get a list of all of the null runs
DIR = '/home/lindb/teakettle/postmoran/chloro/null/outfiles/'
R <- list()
for (f in list.files(DIR,full.names=T,pattern='.RDS')){
    r <- readRDS(f)
    R <- append(R,list(r))
    names(R)[len(R)] <- as.character(len(R))
}
r <- data.frame(sauto$r)
names(r) <- 'r'
R <- append(R,list(r))
names(R)[1000] <- as.character(1000)

# get a list for each distance class, appending values from 1000 null runs
Rm <- list()
for (i in 1:173){
    print(i)
    lst <- c()
    for (j in 1:len(R)){
        lst <- c(lst, R[[as.character(j)]][i,'r'])
    }
    Rm <- append(Rm,list(lst))
}

# get 2.5th and 97.5th quantiles
null <- list()
for (i in 1:len(Rm)){
    low = sort(Rm[[i]])[25]
    high = sort(Rm[[i]])[975]
    null <- append(null,list(c(low,high)))
}


filE = fname(getwd(),'chloro_null.RDS')
saveRDS(null,filE)
filE = fname(getwd(),'chloro_R.RDS')
saveRDS(R,filE)
filE = fname(getwd(),'chloro_Rm.RDS')
saveRDS(Rm,filE)
'''
filE = '/home/lindb/teakettle/postmoran/chloro/combine_R_vector_chloro_geno.R'
with open(filE,'w') as o:
    o.write("%s" % text)
o.close()

### plot spatial autocorrelation
### put into R
```R
source('~/imports.R')
cd('~/Downloads/')

### load Rm, R, and sauto
# null <- readRDS('/home/lindb/teakettle/postmoran/nuclear/nuclear_null.RDS')
# sauto <- readRDS('/home/lindb/teakettle/postmoran/nuclear/final/sauto.RDS')

getrds()
null <- chloro_null

# get quantiles for each distance class
lows <- c()
his <- c()
for (i in 1:len(null)){
	lows <- c(lows,null[[i]][1])
	his <- c(his,null[[i]][2])
}

plot(sauto$bin,sauto$r,type="l",xlab="Distance (meters)",ylab="r",ylim=c(-0.13,0.13),xaxt="n")
xtick <- seq(0,max(sauto$bin),100)
axis(side=1,at=xtick)
polygon(c(rev(sauto$bin),sauto$bin),c(rev(his),lows),col='grey75',border=FALSE)
lines(sauto$bin,sauto$r,lwd=2)
lines(sauto$bin,lows,col="red",lty=2)
lines(sauto$bin,his,col="red",lty=2)
```

# Spatial autocor by treatment - chloro only

### combine treatment ignore inter-plot distances
### put into R
```R
source('~/imports.R')
library(PopGenReport)
cd('~/fragment/postmoran/final')

### load dependencies
getrds()
popgen.tx <- getdf(treatment='tx')
nuc = colnames(popgen.tx)[2:5]
chloro = colnames(popgen.tx)[6:8]
popgen.tx <- popgen.tx[,c('pop',chloro)]
head(popgen.tx)

### get a dataframe for each plot
plots <- uni(popgen.tx$pop)
dfs <- c()
for (plot in plots){
    df <- popgen.tx[which(popgen.tx$pop == plot),]
    assign(sprintf("%spop",plot),df$pop)
    df <- df[,chloro]
    p <- plot
    assign(p,df)
}

### get genind objects
geninds <- c()
for (plot in plots){
    df      <- get(plot)
    pops    <- get(sprintf("%spop",plot))
    genind  <- sprintf("gen%s",plot)
    geninds <- c(geninds,genind)
    assign(genind,df2genind(df,ploidy=2,sep=":",pop=pops))
}

### get gd objects
gdnames = c()
for (genind in geninds){
    gdname <- sprintf("gd%s",genind)
    gdnames <- c(gdnames,gdname)
    gd <- gd.smouse(get(genind))
    gd <- as.matrix(gd)
    assign(gdname,gd)
}


### get euclidian distance
head(distmat) # full distmat from 07a_spatial_auto...ipynb
len(which(is.na(distmat)))
simpmoran[,'plot']      <- substr(simpmoran$samp,1,3)
simpmoran[,'treatment'] <- substr(simpmoran$plot,1,2)
simpmoran[,'plot_rep']  <- substr(simpmoran$plot,3,3)

# from full distmat, create smaller distmats based on treatment type
txs <- uni(simpmoran$treatment)
distnames <- c()
for (tx in txs){
    sampnames <- simpmoran[which(simpmoran[,'treatment']==tx),'samp']
    d <- distmat[which(rownames(distmat) %in% sampnames),which(colnames(distmat) %in% sampnames)]
    d <- d[which(rownames(d) %in% rownames(get(sprintf('gdgen%s',tx)))) ,
            which(colnames(d) %in% colnames(get(sprintf('gdgen%s',tx))))]
    print(dim(d))
    distname  <- sprintf('dist%s',tx)
    distnames <- c(distnames,distname)
    assign(distname,d)
}

### get spatial autocorrelation
autos <- c()
autobins <- c()
for (gd in gdnames){
    print(gd)
    gdist <- get(gd)
    tx <- substr(gd,6,7)
    d <- get(sprintf("dist%s",tx))
    bins = ceiling(.max(d)/10)
    binname = sprintf("bin%s",gd)
    assign(binname,bins)
    autobins <- c(autobins,binname)
    auto <- spautocor(gdist,d,bins=bins)
    autoname <- sprintf("auto%s",gd)
    autos <- c(autos,autoname)
    assign(autoname,auto)
}

### save environment
DIR = '~/fragment/postmoran/by_tx/chloro'
if (!(file.exists(DIR))){
    dir.create(DIR)
}
cd(DIR)
rds(,erase=T)
```

In [ ]:
# copy to godel
DIR = '/home/lindb/teakettle/postmoran/by_tx/chloro/final'
if not op.exists(DIR):
    mkdir(DIR)
# !scp ~/fragment/postmoran/by_tx/chloro/*RDS godel:/home/lindb/teakettle/postmoran/by_tx/chloro/final

### multiprocess null confidence intervals

In [ ]:
dirs = ['/home/lindb/teakettle/postmoran/by_tx/chloro/bc',
        '/home/lindb/teakettle/postmoran/by_tx/chloro/uc',
        '/home/lindb/teakettle/postmoran/by_tx/chloro/un',
        '/home/lindb/teakettle/postmoran/by_tx/chloro/us']
for d in dirs:
    if not op.exists(d):
        mkdir(d)

In [ ]:
# make some dirs
for d in dirs:
    if not 'final' in d:
        dnames = []
        dnames.append(op.join(d,'outfiles'))
        dnames.append(op.join(d,'rfiles'))
        dnames.append(op.join(d,'shfiles'))
        dnames.append(op.join(d,'null'))
        for D in dnames:
            if not op.exists(D):
                mkdir(D)

In [ ]:
# get plot IDs
txs = []
for d in dirs:
    if not 'final' in d:
        txs.append(op.basename(d))
txs

In [ ]:
# write scripts per tx to get 999 null values of spatial autocorrelation
for tx in txs:
    for i in range(999):
        text = '''
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/by_tx/chloro/final/')

# load dependencies
getrds()

spautocor <- function (gen.m, eucl.m, shuffle = FALSE, bins = 10) 
{
    gd <- gen.m
    ed <- eucl.m
    if (shuffle == TRUE) {
        gdd <- as.dist(gd)
        gdsample <- sample(1:length(gdd), length(gdd))
        gd[lower.tri(gd)] <- gdd[gdsample]
        gd[upper.tri(gd)] <- gdd[gdsample]
        diag(gd) <- 0
    }
    cdmat <- function(gd) {
        dimen <- nrow(gd)
        sgd <- sum(gd, na.rm = TRUE)
        cscd <- matrix(colSums(gd, na.rm = TRUE), dimen, dimen)
        rscd <- matrix(rowSums(gd, na.rm = TRUE), dimen, dimen, 
            byrow = TRUE)
        cd <- 0.5 * (-gd + 1/dimen * (cscd + rscd) - 1/dimen^2 * 
            (sgd))
        cd
    }
    cd <- cdmat(gd)
    ed[upper.tri(ed)] <- NA
    diag(ed) <- NA
    r <- NA
    distance <- NA
    N <- NA
    steps <- signif(diff(range(ed, na.rm = TRUE))/bins, 4)
    for (d in 1:bins) {
        index <- which(ed <= d * steps & ed > (d - 1) * steps, 
            arr.ind = TRUE)
        cx <- sum(cd[index])
        cxii <- sum(diag(cd)[index[, 1]])
        cxjj <- sum(diag(cd)[index[, 2]])
        r[d] <- 2 * cx/(cxii + cxjj)
        distance[d] <- steps * d
        N[d] <- length(index)
    }
    if (shuffle == FALSE) 
        res <- data.frame(bin = distance, N = N, r = r)
    else res <- data.frame(r = r)
    res
}

# get null value
gdmat <- get('%s')
dmat  <- get('%s')
bins  <- get('%s')
shuff <- spautocor(gdmat,dmat,bins=bins,shuffle=T)
filE = sprintf('/home/lindb/teakettle/postmoran/by_tx/chloro/%s/outfiles/shuff_%%s.RDS',sprintf('%%03d',%s))
saveRDS(shuff,filE)
    ''' % ('gdgen%s' % tx,
           'dist%s' % tx,
           'bingdgen%s' % tx,
           tx,
           i)
        filE = '/home/lindb/teakettle/postmoran/by_tx/chloro/%s/rfiles/null_%s.R' % (tx,str(i).zfill(3))
        with open(filE,'w') as o:
            o.write("%s" % text)
        o.close()

In [ ]:
# gather r files, make sure there are 999 for each plot
rfiles = []
for p in txs:
    DIR = '/home/lindb/teakettle/postmoran/by_tx/chloro/%s/rfiles' % p
    assert(op.exists(DIR))
    [rfiles.append(op.join(DIR,f)) for f in ls(DIR)]
    print p
    print len([f for f in ls(DIR)])
rfiles = sorted(rfiles)
len(rfiles)

In [ ]:
# how many r files per sh file
ceil = math.ceil(len(rfiles)/195)
ceil

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/by_tx/chloro/shfiles'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
# get .R files
rfiles = [] # redundant so I don't use wrong rfiles
for p in txs:
    DIR = '/home/lindb/teakettle/postmoran/by_tx/chloro/%s/rfiles' % p
    assert(op.exists(DIR))
    [rfiles.append(op.join(DIR,f)) for f in ls(DIR)]
    print p
    print len([f for f in ls(DIR)])
rfiles = sorted(rfiles)

# write sh files
fcount  = 0
shcount = 0
rcount  = 0
for f in rfiles:
    num = str(shcount).zfill(3)
    if rcount == 0:
        text = '''#!/bin/bash
#$ -N shuff%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s
''' % (num,
       op.dirname(f),
       f)
    else:
        newtext = '''
cd %s
R --no-save < %s
''' % (op.dirname(f),
       f)
        text = text + newtext
    fcount += 1
    rcount += 1
    
    if rcount == ceil or fcount == len(rfiles):
        filE = '/home/lindb/teakettle/postmoran/by_tx/chloro/shfiles/shuff_%s.sh' % str(shcount).zfill(3)
        with open(filE,'w') as o:
            o.write("%s" % text)
        rcount = 0
        shcount += 1

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/by_tx/chloro/shfiles/'
shfiles = fs(DIR)
cd(DIR)
!ls *sh | parallel qsub {}

In [ ]:
# check on progress
outfiles = []
for p in txs:
    DIR = '/home/lindb/teakettle/postmoran/by_tx/chloro/%s/outfiles' % p
    assert op.exists(DIR)
    [outfiles.append(f) for f in ls(DIR)]
len(outfiles)

In [ ]:
# combine null runs into a couple objects
for tx in txs:
    text = '''
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/by_tx/chloro/final')
getrds()
tx = '%s'

#sauto <- readRDS('/home/lindb/teakettle/postmoran/nuclear/final/sauto.RDS')
sauto <- get(sprintf('autogdgen%%s',tx))

# get a list of all of the null runs
DIR = sprintf('/home/lindb/teakettle/postmoran/by_tx/chloro/%%s/outfiles',tx)
R <- list()
for (f in list.files(DIR,full.names=T,pattern='.RDS')){
    r <- readRDS(f)
    R <- append(R,list(r))
    names(R)[len(R)] <- as.character(len(R))
}
r <- data.frame(sauto$r)
names(r) <- 'r'
R <- append(R,list(r))
names(R)[1000] <- as.character(1000)

# get a list for each distance class, appending values from 1000 null runs
Rm <- list()
bins <- get(sprintf('bingdgen%%s',tx))
for (i in 1:bins){
    print(i)
    lst <- c()
    for (j in 1:len(R)){
        lst <- c(lst, R[[as.character(j)]][i,'r'])
    }
    Rm <- append(Rm,list(lst))
}

# get 2.5th and 97.5th quantiles
null <- list()
for (i in 1:len(Rm)){
    low = sort(Rm[[i]])[25]
    high = sort(Rm[[i]])[975]
    null <- append(null,list(c(low,high)))
}

### save environment
cd(sprintf('/home/lindb/teakettle/postmoran/by_tx/chloro/%%s',tx))
filE = fname(getwd(),sprintf('%%s_full_geno_null.RDS',tx))
saveRDS(null,filE)
filE = fname(getwd(),sprintf('%%s_full_geno_R.RDS',tx))
saveRDS(R,filE)
filE = fname(getwd(),sprintf('%%s_full_geno_Rm.RDS',tx))
saveRDS(Rm,filE)

''' % (tx)
    filE = '/home/lindb/teakettle/postmoran/by_tx/chloro/%s/%s_combine_R_vector_full_geno.R' % (tx,tx)
    with open(filE,'w') as o:
        o.write("%s" % text)
    o.close()

In [ ]:
rfiles = []
for tx in txs:
    f = '/home/lindb/teakettle/postmoran/by_tx/chloro/%s/%s_combine_R_vector_full_geno.R' % (tx,tx)
    rfiles.append(f)

shfiles = []
for f in rfiles:
    tx = op.basename(op.dirname(f))
    text ='''#!/bin/bash
#$ -N comb%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s

''' % (tx,op.dirname(f),f)
    DIR = '/home/lindb/teakettle/postmoran/by_tx/chloro/shfiles/%s' % tx
    if not op.exists(DIR):
        mkdir(DIR)
    filE = op.join(DIR,'%s.sh' % tx)
    shfiles.append(filE)
    with open(filE,'w') as o:
        o.write("%s" % text)

In [ ]:
for tx in txs:
    f = '/home/lindb/teakettle/postmoran/by_tx/chloro/shfiles/%s/%s.sh' % (tx,tx)
    assert op.exists(f)
    cd(op.dirname(f))
    !qsub $f

In [ ]:
# scp these files to computer
files = [f for f in fs('/home/lindb/teakettle/postmoran/by_tx/chloro/final/') if 'autogdgen' in f]
[files.append(f) for f in fs('/home/lindb/teakettle/postmoran/by_tx/chloro/final/') if 'bingdgen' in f]
for tx in txs:
    f = '/home/lindb/teakettle/postmoran/by_tx/chloro/%s/%s_full_geno_null.RDS' % (tx,tx)
    assert op.exists(f)
    files.append(f)
files

### put into R
```R
source('~/imports.R')
library(PopGenReport)
library(adegenet)
cd('~/Downloads/by_tx_chloro')
getrds()
txs <- c('bc','uc','un','us')

Lows <- list()
His  <- list()
for (tx in txs){
    null <- get(sprintf('%s_full_geno_null',tx))
    h <- c()
    l <- c()
    for (i in 1:len(null)){
        h <- c(h,null[[i]][1])
        l <- c(l,null[[i]][2])
    }
    Lows <- append(Lows,list(l))
    His  <- append(His,list(h))
    names(Lows)[len(Lows)] <- tx
    names(His)[len(His)]   <- tx
}
```

In [ ]:
# print out text to paste into R
for tx in txs:
    text = '''
########
tx = '%s'
his  <- His[[sprintf('%%s',tx)]]
his[which(is.na(his))] <- 0
lows <- Lows[[sprintf('%%s',tx)]]
lows[which(is.na(lows))] <- 0
sauto <- get(sprintf('autogdgen%%s',tx))
sauto[which(is.na(sauto$r)),'r'] <- 0
cutoff <- which(sauto$bin > 325)
dev.new(width=3.25,height=3.25)
par(cex.lab=1,cex.axis=1)
plot(sauto$bin,sauto$r,
    type="l",
    xlab="",
    ylab="",
    ylim=c(-0.2,0.13),
    xaxt="n",
    main="")
xtick <- seq(0,max(sauto$bin),200)
axis(side=1,at=xtick)
#polygon(c(rev(sauto$bin[-cutoff]),sauto$bin[-cutoff]),c(rev(his[-cutoff]),lows[-cutoff]),col='grey75',border=FALSE)
#lines(sauto$bin,sauto$r,lwd=2)
#lines(sauto$bin[-cutoff],lows[-cutoff],col="red",lty=2)
#lines(sauto$bin[-cutoff],his[-cutoff],col="red",lty=2)
polygon(c(rev(sauto$bin),sauto$bin),c(rev(his),lows),col='grey75',border=FALSE)
lines(sauto$bin,sauto$r,lwd=2)
lines(sauto$bin,lows,col="red",lty=2)
lines(sauto$bin,his,col="red",lty=2)
####
dev.new(width=10,height=10)
par(cex.lab=1.5,cex.axis=1.5)
plot(sauto$bin,sauto$r,
    type="l",
    xlab="Distance (meters)",
    ylab="r",
    ylim=c(-0.13,0.13),
    xlim=c(0,300),
    xaxt="n",
    xaxt='n') #    main=sprintf('%%s',tx),
xtick <- seq(0,max(sauto$bin),25)
axis(side=1,at=xtick,FALSE)
axis(side=1,at=xtick[xtick %%%% 50 ==0],labels=xtick[xtick %%%% 50 ==0])
polygon(c(rev(sauto$bin),sauto$bin),c(rev(his),lows),col=grey.colors(15)[5],border=FALSE)
lines(sauto$bin,sauto$r,lwd=2)
lines(sauto$bin,lows,col="red",lty=2)
lines(sauto$bin,his,col="red",lty=2)
abline(v=xtick,col='grey',lwd=0.5)
########
''' % tx
    print text